## Examining Student and Online Perceptions of LSE

### Part 1: Introduction

Before arriving at LSE, many students likely hear of the university's low student satisfaction rates. These impressions often gain traction on online forums such as Reddit, where anecdotal accounts tend to amplify perceptions of dissatisfaction. Additionally, multiple university ranking websites also present LSE as having a comparatively low student satisfaction rate. Therefore, it is worth exploring this topic, to understand how "severe" the problem is and whether the data fully aligns with the online perception of the university.

We can explore the statement above through a series of focused questions:
- How does LSE student satisfaction compare to other universities?
- How does student satisfaction within LSE vary by degree?
- How has LSE student satisfaction evolved over the last few years (2020-2023)?
- How is LSE perceived by people online? How does it compare with other universities's perceptions online?

It is worth considering what is meant by student satisfaction. Many data sources such as the National Student Survey break down student's views into categories such as teaching, course content, course organisation and assessment. Meanwhile, online the focus tends to be on social opportunities, as well as societies. As "student satisfaction" is multifaceted we will be breaking down each question into the specific type of student satisfaction we are discussing, to avoid confusion. The NSS is generally better suited towards 'academic' related definitions of student satisfaction, such as student's views on the quality of the course, the teaching etc. whilst Reddit and online platforms likely better represent student's thoughts on their social lives at certain universities.




### Part 2: Data acquisition
- Go to 'Data_acquisition.ipynb' file

### Part 3: Data Preparation and Exploration

In [33]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pickle
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from scipy.stats import linregress

### NSS data

We can answer many of the questions through the NSS. The NSS questions students on multiple different categories, such as teaching quality, course quality and assessment quality. Most websites ignore this distinction and simply focus on one of these metrics such as teaching quality (such as the Complete University Guide), however, we wanted to make the distinction between categories clearer. As there are multiple categories, we create multiple dataframes so that visualisation is easier. Another important thing to consider is the response to population size; if it is high it means the measure of student satisfaction is more likely to accurately represent the real student body, compared to if it is low, when the results are less robust. This is the reason why many online displays of these metrics leave out universities like the University of Oxford, where a hefty proportion of the students boycott the NSS. However, we decided to still compare Oxford to LSE, whilst acknowledging this potential issue with the data.

In [3]:
file_path = './data/univ_df.pkl'

# Load the dictionary from the pickle file
with open(file_path, 'rb') as pickle_file:
    univ_df = pickle.load(pickle_file)

#refresher of the universities surveyed
for key in univ_df.keys():
    print(key)

LSE
Oxford
UCL
Birmingham
Edinburgh
Glasgow
Imperial
KCL
Manchester
Norwich
Strathclyde
Warwick


Below, we create dataframes representing the differences in student opinions on teaching quality, course quality, assessment quality, support quality, as well as course organisation quality. We considered aggregating every metric into one 'overall satisfaction' score, however, feel that would not take into account the fact that some may view certain criterion as deserving more weight; for example, learning opportunities may be valued much more than assessment quality. These themes are represented on the NSS and are calculated through averaging student responses to questions that can be grouped into one larger theme.


In [4]:
main_dict={}

for name,uni_df in univ_df.items():
    
    #We measure 'All Subjects' at this point as we don't want to differentiate by Subject
    #The last 7 rows of the dataframe refer to the Themes we are interested in
    df=uni_df[(uni_df["Level of study"]=='All undergraduates') & (uni_df["Subject level"]=="All subjects")& (uni_df['Level of study']=='All undergraduates')][-7:]
    response_ratio=round(df.iloc[0]['Responses']/df.iloc[0]['Population'],2)

    #Teaching Scores
    teach_positivity=round(df.iloc[0]['Positivity measure (%)'],1)

    #Learning Opportunities
    learn_positivity=round(df.iloc[1]['Positivity measure (%)'],1)
    
    #Assessment and feedback
    assess_positivity=round(df.iloc[2]['Positivity measure (%)'],1)
   
    #Academic Support
    support_positivity=round(df.iloc[3]['Positivity measure (%)'],1)
 
    # Organisation and Management
    organisation_positivity=round(df.iloc[4]['Positivity measure (%)'],1)

    # Learning Resources
    resource_positivity=round(df.iloc[5]['Positivity measure (%)'],1)

    # Student Voice
    voice_positivity=round(df.iloc[6]['Positivity measure (%)'],1)
  
    main_dict[name]={'Teaching': teach_positivity, 'Learning Opportunities': learn_positivity, 'Assessment and Feedback': assess_positivity, 
                     'Academic Support': support_positivity, 'Organisation and Management': organisation_positivity,
                    'Learning Resources': resource_positivity, 'Student Voice': voice_positivity, 'Response Ratio': response_ratio}
    



In [5]:
overall_df=pd.DataFrame(main_dict)
overall_df=overall_df.transpose()
overall_df

,Teaching,Learning Opportunities,Assessment and Feedback,Academic Support,Organisation and Management,Learning Resources,Student Voice,Response Ratio
LSE,86.5,79.4,71.1,87.4,80.7,87.1,67.2,0.67
Oxford,93.1,82.1,74.1,90.4,67.6,92.2,61.9,0.50
UCL,84.4,79.4,68.6,84.1,74.0,89.4,73.1,0.72
Birmingham,84.1,79.2,68.2,81.4,72.5,87.2,64.4,0.69
Edinburgh,85.2,75.5,63.4,81.3,69.1,85.7,65.6,0.65
Glasgow,98.1,96.2,95.6,100.0,98.1,94.9,90.6,0.71
Imperial,89.3,85.1,68.7,86.6,73.5,91.2,76.7,0.72
KCL,83.3,77.7,67.9,80.6,66.6,85.1,66.5,0.70
Manchester,81.7,76.9,67.9,81.3,70.2,81.9,64.4,0.74
Norwich,82.6,79.1,81.6,87.4,68.7,85.5,72.0,0.82


This much data is a little overwhelming when trying to compare universities and it would be nice if we could see the ranking for different categories. We will be able to better see differences in the Data Visualisation section, however, we decided to look at individual categories closer by slicing our DataFrame. 


In [6]:
teaching_df = overall_df.iloc[:, [0, 7]]
teaching_df.sort_values(by='Teaching', ascending=False)

,Teaching,Response Ratio
Glasgow,98.1,0.71
Strathclyde,94.1,0.67
Oxford,93.1,0.50
Imperial,89.3,0.72
Warwick,87.1,0.72
LSE,86.5,0.67
Edinburgh,85.2,0.65
UCL,84.4,0.72
Birmingham,84.1,0.69
KCL,83.3,0.70


In [7]:
learning_df = overall_df.iloc[:, [1, 7]]
learning_df.sort_values(by='Learning Opportunities', ascending=False)

,Learning Opportunities,Response Ratio
Glasgow,96.2,0.71
Strathclyde,92.0,0.67
Warwick,90.3,0.72
Imperial,85.1,0.72
Oxford,82.1,0.50
LSE,79.4,0.67
UCL,79.4,0.72
Birmingham,79.2,0.69
Norwich,79.1,0.82
KCL,77.7,0.70


In [8]:
assess_df = overall_df.iloc[:, [2, 7]]
assess_df.sort_values(by='Assessment and Feedback', ascending=False)

,Assessment and Feedback,Response Ratio
Glasgow,95.6,0.71
Warwick,91.0,0.72
Strathclyde,89.3,0.67
Norwich,81.6,0.82
Oxford,74.1,0.50
LSE,71.1,0.67
Imperial,68.7,0.72
UCL,68.6,0.72
Birmingham,68.2,0.69
KCL,67.9,0.70


In [9]:
support_df = overall_df.iloc[:, [3, 7]]
support_df.sort_values(by='Academic Support', ascending=False)

,Academic Support,Response Ratio
Glasgow,100.0,0.71
Strathclyde,97.1,0.67
Oxford,90.4,0.50
LSE,87.4,0.67
Norwich,87.4,0.82
Imperial,86.6,0.72
UCL,84.1,0.72
Birmingham,81.4,0.69
Edinburgh,81.3,0.65
Manchester,81.3,0.74


In [10]:
org_df = overall_df.iloc[:, [4, 7]]
org_df.sort_values(by='Organisation and Management', ascending=False)

,Organisation and Management,Response Ratio
Glasgow,98.1,0.71
Strathclyde,87.3,0.67
LSE,80.7,0.67
Warwick,74.2,0.72
UCL,74.0,0.72
Imperial,73.5,0.72
Birmingham,72.5,0.69
Manchester,70.2,0.74
Edinburgh,69.1,0.65
Norwich,68.7,0.82


In [11]:
resource_df = overall_df.iloc[:, [5, 7]]
resource_df.sort_values(by='Learning Resources', ascending=False)

,Learning Resources,Response Ratio
Glasgow,94.9,0.71
Oxford,92.2,0.50
Strathclyde,92.0,0.67
Imperial,91.2,0.72
UCL,89.4,0.72
Birmingham,87.2,0.69
LSE,87.1,0.67
Edinburgh,85.7,0.65
Norwich,85.5,0.82
KCL,85.1,0.70


In [12]:
voice_df = overall_df.iloc[:, [6, 7]]
voice_df.sort_values(by='Student Voice', ascending=False)

,Student Voice,Response Ratio
Glasgow,90.6,0.71
Strathclyde,77.5,0.67
Imperial,76.7,0.72
UCL,73.1,0.72
Norwich,72.0,0.82
Warwick,69.9,0.72
LSE,67.2,0.67
KCL,66.5,0.70
Edinburgh,65.6,0.65
Birmingham,64.4,0.69


The results were quite surprising to us as LSE appeared to have decently satisfied students in multiple categories, most notable being Course Organisation quality, where LSE placed third. In fact, out of all of the London universities LSE appeared to perform as one of the best. We decided to investigate the disparities between universities further in the Visualisation section.

Another research question involved investigating how student satisfaction varied by degree at the LSE. We look at all of the subjects first to get an idea of the data.

In [13]:
lse_subjectdat=univ_df['LSE']
lse_subjectdat['Subject'].value_counts(dropna=False)

Law                                                408
Psychology                                         272
Mathematical sciences                              272
Geography, earth and environmental studies         272
Economics                                          272
Politics                                           272
Business and management                            272
Language and area studies                          136
Human geography                                    136
Philosophy                                         136
Philosophy and religious studies                   136
History                                            136
History and archaeology                            136
Historical, philosophical and religious studies    136
Asian studies                                      136
Languages and area studies                         136
Management studies                                 136
Accounting                                         136
Finance   

Many of these subjects overlap due to the way the NSS is organised. Essentially, subjects are grouped into certain levels under the Common Aggregation Hierarchy; the more broad a subject is the lower the level. For example, at CAH1 you could get 'Geography, earth and environmental studies' whilst at CAH3 you can get 'Human geography.' For this project, we have decided to only analyse subjects at the level of CAH2. Though this does lose some núance (e.g. Mathematics and Statistics are grouped into one 'Mathematical Sciences') it makes the data easier to interpret, as the differences between groups may appear more evident.  

In [14]:
lse_subjectdat=lse_subjectdat[lse_subjectdat['Subject level']=='CAH2']
lse_subjectdat['Subject'].value_counts()


Psychology                                    136
Mathematical sciences                         136
Sociology, social policy and anthropology     136
Economics                                     136
Politics                                      136
Law                                           136
Business and management                       136
Languages and area studies                    136
History and archaeology                       136
Philosophy and religious studies              136
Geography, earth and environmental studies    136
Name: Subject, dtype: int64

We then calculate the specific positivity scores for each subject.

The data differentiates between students with a First and 'all undergraduates', but we could not see any difference between the data collected for both values (i.e. the rows were identical with the one difference being some had 'First') which led us to believe there was no real need to display this distinction. As, such we only analysed rows with Level of study as 'All undergraduates'. 

For the academic subjects, we decided to also display the standard deviation, as we felt it was important to determine how divided student opinion was in certain subjects. Additionally, some cohorts are much smaller than others (e.g. Psychology had only 30 respondents) so we also displayed this.

In [15]:
teach_dict={}
learn_dict={}
assess_dict={}
support_dict={}
organisation_dict={}
resource_dict={}
voice_dict={}
overall_subject={}
for name in lse_subjectdat['Subject'].unique():
    
    df=lse_subjectdat[(lse_subjectdat['Subject']==name) & (lse_subjectdat['Level of study']=='All undergraduates')][-7:]
    response_ratio=round(df.iloc[0]['Responses']/df.iloc[0]['Population'],2)
    
    
    #Teaching Scores
    teach_positivity=df.iloc[0]['Positivity measure (%)']
    sd=df.iloc[0]['Standard deviation']
    respondents=df.iloc[0]['Responses']
    teach_dict[name]={'Positivity Measure(%)':teach_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}
    
    #Learning Opportunities
    learn_positivity=df.iloc[1]['Positivity measure (%)']
    sd=df.iloc[1]['Standard deviation']
    respondents=df.iloc[1]['Responses']
    learn_dict[name]={'Positivity Measure(%)':learn_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}

    
    #Assessment and feedback
    assess_positivity=df.iloc[2]['Positivity measure (%)']
    sd=df.iloc[2]['Standard deviation']
    respondents=df.iloc[2]['Responses']
    assess_dict[name]={'Positivity Measure(%)':assess_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}

    #Academic Support
    support_positivity=df.iloc[3]['Positivity measure (%)']
    sd=df.iloc[3]['Standard deviation']
    respondents=df.iloc[3]['Responses']
    support_dict[name]={'Positivity Measure(%)':support_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}

    # Organisation and Management
    organisation_positivity=df.iloc[4]['Positivity measure (%)']
    sd=df.iloc[4]['Standard deviation']
    respondents=df.iloc[4]['Responses']
    organisation_dict[name]={'Positivity Measure(%)':organisation_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}

    # Learning Resources
    resource_positivity=df.iloc[5]['Positivity measure (%)']
    sd=df.iloc[5]['Standard deviation']
    respondents=df.iloc[5]['Responses']
    resource_dict[name]={'Positivity Measure(%)':resource_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}

    # Student Voice
    voice_positivity=df.iloc[6]['Positivity measure (%)']
    sd=df.iloc[6]['Standard deviation']
    respondents=df.iloc[6]['Responses']
    voice_dict[name]={'Positivity Measure(%)':voice_positivity, 'Standard Deviation': sd, 'Response Ratio': response_ratio, 'Number of Respondents': int(respondents)}

    #overall
    overall_subject[name]={'Teaching': teach_positivity, 'Learning Opportunities': learn_positivity, 'Assessment and Feedback': assess_positivity, 
                     'Academic Support': support_positivity, 'Organisation and Management': organisation_positivity,
                    'Learning Resources': resource_positivity, 'Student Voice': voice_positivity, 'Response Ratio': response_ratio}
    
    
    


In [16]:
overall_subjectdf=pd.DataFrame(overall_subject)
overall_subjectdf=overall_subjectdf.transpose()
overall_subjectdf

,Teaching,Learning Opportunities,Assessment and Feedback,Academic Support,Organisation and Management,Learning Resources,Student Voice,Response Ratio
Psychology,99.2,96.0,89.3,100.0,96.7,93.9,100.0,1.00
Mathematical sciences,72.8,69.3,58.6,76.9,70.8,84.6,59.6,0.63
"Sociology, social policy and anthropology",90.1,82.6,77.5,89.8,80.9,89.8,72.9,0.71
Economics,84.1,75.7,68.2,86.5,81.9,85.4,64.4,0.65
Politics,89.2,79.3,69.2,87.9,78.2,88.4,63.6,0.73
Law,93.3,82.3,79.5,90.0,76.7,87.7,67.5,0.59
Business and management,88.0,82.7,70.3,88.0,85.6,86.8,69.6,0.66
Languages and area studies,93.3,89.8,92.4,100.0,84.5,86.0,75.6,0.77
History and archaeology,90.3,84.6,78.1,91.2,85.6,88.5,71.4,0.74
Philosophy and religious studies,90.4,83.9,79.1,93.1,83.8,88.3,73.8,0.68


In [18]:
teach_df=pd.DataFrame(teach_dict)
print('Student Views on Teaching Quality by Subject') #a higher percentage means more students think positively of that factor
teach_df=teach_df.transpose()
teach_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Teaching Quality by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Psychology,99.2,5.1,1.00,30.0
Law,93.3,2.8,0.59,118.0
Languages and area studies,93.3,10.5,0.77,7.0
Philosophy and religious studies,90.4,4.7,0.68,36.0
History and archaeology,90.3,2.8,0.74,97.0
"Sociology, social policy and anthropology",90.1,3.0,0.71,117.0
Politics,89.2,2.4,0.73,173.0
Business and management,88.0,2.4,0.66,210.0
Economics,84.1,2.1,0.65,297.0
"Geography, earth and environmental studies",78.6,5.1,0.70,51.0


In [19]:
learn_df=pd.DataFrame(learn_dict)
print('Student Views on Learning Opportunities by Subject')
learn_df=learn_df.transpose()
learn_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Learning Opportunities by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Psychology,96.0,6.0,1.00,30.0
Languages and area studies,89.8,12.9,0.77,7.0
History and archaeology,84.6,3.7,0.74,97.0
Philosophy and religious studies,83.9,6.0,0.68,36.0
Business and management,82.7,2.6,0.66,210.0
"Sociology, social policy and anthropology",82.6,3.5,0.71,117.0
Law,82.3,3.5,0.59,118.0
Politics,79.3,3.0,0.73,173.0
Economics,75.7,2.4,0.65,297.0
"Geography, earth and environmental studies",75.3,5.5,0.70,51.0


In [20]:
print('Student Views on Assessment Quality by Subject')
assess_df=pd.DataFrame(assess_dict)
assess_df=assess_df.transpose()
assess_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Assessment Quality by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Languages and area studies,92.4,13.1,0.77,7.0
Psychology,89.3,6.9,1.00,30.0
Law,79.5,3.8,0.59,118.0
Philosophy and religious studies,79.1,6.5,0.68,36.0
History and archaeology,78.1,4.0,0.74,97.0
"Sociology, social policy and anthropology",77.5,3.7,0.71,117.0
Business and management,70.3,2.9,0.66,210.0
Politics,69.2,3.3,0.73,173.0
Economics,68.2,2.6,0.65,297.0
"Geography, earth and environmental studies",64.3,6.2,0.70,51.0


In [21]:
print('Student Views on Academic Support Quality by Subject')
support_df=pd.DataFrame(support_dict)
support_df=support_df.transpose()
support_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Academic Support Quality by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Psychology,100.0,5.3,1.00,30.0
Languages and area studies,100.0,8.4,0.77,7.0
Philosophy and religious studies,93.1,4.7,0.68,36.0
History and archaeology,91.2,2.9,0.74,97.0
Law,90.0,3.1,0.59,118.0
"Sociology, social policy and anthropology",89.8,3.2,0.71,117.0
Business and management,88.0,2.3,0.66,210.0
Politics,87.9,2.5,0.73,173.0
Economics,86.5,2.0,0.65,297.0
"Geography, earth and environmental studies",82.8,4.9,0.70,51.0


In [22]:
print('Student Views on Course Organisation Quality by Subject')
organisation_df=pd.DataFrame(organisation_dict)
organisation_df=organisation_df.transpose()
organisation_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Course Organisation Quality by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Psychology,96.7,6.1,1.00,30.0
Business and management,85.6,2.5,0.66,210.0
History and archaeology,85.6,3.8,0.74,97.0
Languages and area studies,84.5,14.8,0.77,7.0
Philosophy and religious studies,83.8,6.2,0.68,36.0
Economics,81.9,2.2,0.65,297.0
"Sociology, social policy and anthropology",80.9,3.8,0.71,117.0
Politics,78.2,3.1,0.73,173.0
Law,76.7,3.8,0.59,118.0
"Geography, earth and environmental studies",74.7,5.7,0.70,51.0


In [23]:
print('Student Views on Resources by Subject')
resource_df=pd.DataFrame(resource_dict)
resource_df=resource_df.transpose()
resource_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Resources by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Psychology,93.9,5.2,1.00,30.0
"Sociology, social policy and anthropology",89.8,2.9,0.71,117.0
History and archaeology,88.5,3.1,0.74,97.0
Politics,88.4,2.4,0.73,173.0
Philosophy and religious studies,88.3,5.2,0.68,36.0
Law,87.7,3.0,0.59,117.0
Business and management,86.8,2.3,0.66,209.0
Languages and area studies,86.0,12.2,0.77,7.0
Economics,85.4,1.9,0.65,294.0
Mathematical sciences,84.6,2.8,0.63,133.0


In [24]:
print('Student Views on Student Voice by Subject')
voice_df=pd.DataFrame(voice_dict)
voice_df=voice_df.transpose()
voice_df.sort_values(by='Positivity Measure(%)', ascending=False)

Student Views on Student Voice by Subject


,Positivity Measure(%),Standard Deviation,Response Ratio,Number of Respondents
Psychology,100.0,6.2,1.00,30.0
Languages and area studies,75.6,15.8,0.77,7.0
Philosophy and religious studies,73.8,7.1,0.68,36.0
"Sociology, social policy and anthropology",72.9,4.1,0.71,117.0
History and archaeology,71.4,4.4,0.74,96.0
Business and management,69.6,3.0,0.66,209.0
Law,67.5,4.2,0.59,117.0
Economics,64.4,2.6,0.65,294.0
Politics,63.6,3.5,0.73,171.0
Mathematical sciences,59.6,3.9,0.63,134.0


Side by side bar chart to visualise 'overall_subjectdf'
- Factors that we decided to consider from the above dataframe based on relevance to the question: Teaching, Learning Opportunities, Academic Support, Organisational Management
- Each bar also includes data on standard deviation, response ratio and number of respondants for each measure


In [ ]:
import plotly.graph_objects as go

In [ ]:
teach_df = teach_df.reset_index()
learn_df = learn_df.reset_index()
support_df = support_df.reset_index()
organisation_df = organisation_df.reset_index()

In [36]:
merged_df = overall_subjectdf.copy()
merged_df = merged_df.reset_index()

#merge the standard deviation in individual data frames to overall_subject.df on Subject 'index'
merged_df = merged_df.merge(teach_df[['index', 'Standard Deviation']],on='index',how='left',).rename(columns={'Standard Deviation': 'Teaching Std Dev'})
merged_df = merged_df.merge(learn_df[['index', 'Standard Deviation']],on='index',how='left',).rename(columns={'Standard Deviation': 'Learning Std Dev'})
merged_df = merged_df.merge(support_df[['index', 'Standard Deviation']],on='index',how='left',).rename(columns={'Standard Deviation': 'Support Std Dev'})
merged_df = merged_df.merge(organisation_df[['index', 'Standard Deviation']],on='index',how='left',).rename(columns={'Standard Deviation': 'Organisation Std Dev'})

#create hover text for standard deviation and response ratio
merged_df['Teaching_hovertext'] = merged_df.apply(lambda row: f'Standard Deviation: {row["Teaching Std Dev"]:.2f}<br>'f'Response Ratio: {row["Response Ratio"]:.2f}',axis=1)
merged_df['Learning_hovertext'] = merged_df.apply(lambda row: f'Standard Deviation: {row["Learning Std Dev"]:.2f}<br>'f'Response Ratio: {row["Response Ratio"]:.2f}',axis=1)
merged_df['Support_hovertext'] = merged_df.apply(lambda row: f'Standard Deviation: {row["Support Std Dev"]:.2f}<br>'f'Response Ratio: {row["Response Ratio"]:.2f}',axis=1)
merged_df['Organisation_hovertext'] = merged_df.apply(lambda row: f'Standard Deviation: {row["Organisation Std Dev"]:.2f}<br>'f'Response Ratio: {row["Response Ratio"]:.2f}',axis=1)

#create the figure
fig = go.Figure()

#add bar traces for each measure
fig.add_trace(go.Bar(x=merged_df['index'],y=merged_df['Teaching'],name='Teaching',hovertext=merged_df['Teaching_hovertext'],hoverinfo='text',))
fig.add_trace(go.Bar(x=merged_df['index'],y=merged_df['Learning Opportunities'],name='Learning Opportunities',hovertext=merged_df['Learning_hovertext'],hoverinfo='text',))
fig.add_trace(go.Bar(x=merged_df['index'],y=merged_df['Academic Support'],name='Academic Support',hovertext=merged_df['Support_hovertext'],hoverinfo='text'))
fig.add_trace(go.Bar(x=merged_df['index'],y=merged_df['Organisation and Management'],name='Organisation and Management',hovertext=merged_df['Organisation_hovertext'],hoverinfo='text',))

#update the layout
fig.update_layout(
    barmode='stack',
    title='Student satisfaction at LSE by degree subject',
    xaxis_title='Subject',
    yaxis_title='Score, 0-100 per measure',
)

#show the figure
fig.show()

The tables surprised us, as we did not expect the Positivity across subjects to vary so much. In particular, it seemed at a first glance, that humanities students seemed more satisfied in general than more quantitative degrees( though Geography was an exception, Mathematical sciences nearly got last place every time.) So we grouped subjects we felt were humanities against the more quantitative subjects. Business and management was classed as Quantitative since it encompassed Finance and Management at the LSE, which both require compulsory mathematics courses, so we chose to include those within the quantitative category. 

It should be noted that this categorisation is largely subjective, as due to the influence of outside options, students from a degree like 'Philosophy, logic and scientific method' could take many courses in Mathematics and essentially have a largely quantitative degree. Still, as an average, these individuals will not take as many quantitative courses.

To aggregate the columns when combining into new rows, we wanted to sum some columns, like number of respondents, but take the mean of others like the Positivity measure.

Most of the code is repetitive, and the gist of it can be understood by looking at the first "for teaching" block of code 

In [63]:
less_quantitative=['Psychology', 'Languages and area studies', 'Philosophy and religious studies', 
                   'Sociology, social policy and anthropology', 'History and archaeology', 'Law', 'Politics',
                  'Geography, earth and environmental studies']
quantitative=['Business and management', 'Economics', 'Mathematical sciences']
dataframes={'Teach':teach_df, 'Learn': learn_df, 'Assess':assess_df, 'Support': support_df, 'Organisation': organisation_df
            ,'Resources': resource_df, 'Voice': voice_df}


#For Teaching
quant_teach_df=teach_df.loc[quantitative]
respondents=quant_teach_df['Number of Respondents'].sum()
response_ratio=round(quant_teach_df['Response Ratio'].mean(),2)
positivity=round(quant_teach_df['Positivity Measure(%)'].mean(),2)
sd=round(quant_teach_df['Standard Deviation'].mean(),2)

lssquant_teach_df=teach_df.loc[less_quantitative]
lssrespondents=lssquant_teach_df['Number of Respondents'].sum()
lssresponse_ratio=round(lssquant_teach_df['Response Ratio'].mean(),2)
lsspositivity=round(lssquant_teach_df['Positivity Measure(%)'].mean(),2)
lssd=round(lssquant_teach_df['Standard Deviation'].mean(),2)

teach_comparison=pd.DataFrame({'Quantitative': {'Positivity Measure(%)':positivity, 'Standard Deviation':sd
                                                , 'Response Ratio': response_ratio, 'Number of Respondents':respondents },
                                'Less Quantitative':{'Positivity Measure(%)':lsspositivity, 'Standard Deviation':lssd
                                                , 'Response Ratio': lssresponse_ratio, 'Number of Respondents':lssrespondents}})

# For Learning
quant_learn_df = learn_df.loc[quantitative]
learn_respondents = quant_learn_df['Number of Respondents'].sum()
learn_response_ratio = round(quant_learn_df['Response Ratio'].mean(), 2)
learn_positivity = round(quant_learn_df['Positivity Measure(%)'].mean(), 2)
learn_sd = round(quant_learn_df['Standard Deviation'].mean(), 2)

lss_quant_learn_df = learn_df.loc[less_quantitative]
lss_learn_respondents = lss_quant_learn_df['Number of Respondents'].sum()
lss_learn_response_ratio = round(lss_quant_learn_df['Response Ratio'].mean(), 2)
lss_learn_positivity = round(lss_quant_learn_df['Positivity Measure(%)'].mean(), 2)
lss_learn_sd = round(lss_quant_learn_df['Standard Deviation'].mean(), 2)

learn_comparison = pd.DataFrame({
    'Quantitative': {'Positivity Measure(%)': learn_positivity, 'Standard Deviation': learn_sd,
                     'Response Ratio': learn_response_ratio, 'Number of Respondents': learn_respondents},
    'Less Quantitative': {'Positivity Measure(%)': lss_learn_positivity, 'Standard Deviation': lss_learn_sd,
                          'Response Ratio': lss_learn_response_ratio, 'Number of Respondents': lss_learn_respondents}
})

# For Assess DataFrame
quant_assess_df = assess_df.loc[quantitative]
assess_respondents = quant_assess_df['Number of Respondents'].sum()
assess_response_ratio = round(quant_assess_df['Response Ratio'].mean(), 2)
assess_positivity = round(quant_assess_df['Positivity Measure(%)'].mean(), 2)
assess_sd = round(quant_assess_df['Standard Deviation'].mean(), 2)

lss_quant_assess_df = assess_df.loc[less_quantitative]
lss_assess_respondents = lss_quant_assess_df['Number of Respondents'].sum()
lss_assess_response_ratio = round(lss_quant_assess_df['Response Ratio'].mean(), 2)
lss_assess_positivity = round(lss_quant_assess_df['Positivity Measure(%)'].mean(), 2)
lss_assess_sd = round(lss_quant_assess_df['Standard Deviation'].mean(), 2)

assess_comparison = pd.DataFrame({
    'Quantitative': {'Positivity Measure(%)': assess_positivity, 'Standard Deviation': assess_sd,
                     'Response Ratio': assess_response_ratio, 'Number of Respondents': assess_respondents},
    'Less Quantitative': {'Positivity Measure(%)': lss_assess_positivity, 'Standard Deviation': lss_assess_sd,
                          'Response Ratio': lss_assess_response_ratio, 'Number of Respondents': lss_assess_respondents}
})


# For Support DataFrame
quant_support_df = support_df.loc[quantitative]
support_respondents = quant_support_df['Number of Respondents'].sum()
support_response_ratio = round(quant_support_df['Response Ratio'].mean(), 2)
support_positivity = round(quant_support_df['Positivity Measure(%)'].mean(), 2)
support_sd = round(quant_support_df['Standard Deviation'].mean(), 2)

lss_quant_support_df = support_df.loc[less_quantitative]
lss_support_respondents = lss_quant_support_df['Number of Respondents'].sum()
lss_support_response_ratio = round(lss_quant_support_df['Response Ratio'].mean(), 2)
lss_support_positivity = round(lss_quant_support_df['Positivity Measure(%)'].mean(), 2)
lss_support_sd = round(lss_quant_support_df['Standard Deviation'].mean(), 2)

support_comparison = pd.DataFrame({
    'Quantitative': {'Positivity Measure(%)': support_positivity, 'Standard Deviation': support_sd,
                     'Response Ratio': support_response_ratio, 'Number of Respondents': support_respondents},
    'Less Quantitative': {'Positivity Measure(%)': lss_support_positivity, 'Standard Deviation': lss_support_sd,
                          'Response Ratio': lss_support_response_ratio, 'Number of Respondents': lss_support_respondents}
})


# For Organisation DataFrame
quant_organisation_df = organisation_df.loc[quantitative]
organisation_respondents = quant_organisation_df['Number of Respondents'].sum()
organisation_response_ratio = round(quant_organisation_df['Response Ratio'].mean(), 2)
organisation_positivity = round(quant_organisation_df['Positivity Measure(%)'].mean(), 2)
organisation_sd = round(quant_organisation_df['Standard Deviation'].mean(), 2)

lss_quant_organisation_df = organisation_df.loc[less_quantitative]
lss_organisation_respondents = lss_quant_organisation_df['Number of Respondents'].sum()
lss_organisation_response_ratio = round(lss_quant_organisation_df['Response Ratio'].mean(), 2)
lss_organisation_positivity = round(lss_quant_organisation_df['Positivity Measure(%)'].mean(), 2)
lss_organisation_sd = round(lss_quant_organisation_df['Standard Deviation'].mean(), 2)

organisation_comparison = pd.DataFrame({
    'Quantitative': {'Positivity Measure(%)': organisation_positivity, 'Standard Deviation': organisation_sd,
                     'Response Ratio': organisation_response_ratio, 'Number of Respondents': organisation_respondents},
    'Less Quantitative': {'Positivity Measure(%)': lss_organisation_positivity, 'Standard Deviation': lss_organisation_sd,
                          'Response Ratio': lss_organisation_response_ratio, 'Number of Respondents': lss_organisation_respondents}
})

# For Resources DataFrame
quant_resources_df = resource_df.loc[quantitative]
resources_respondents = quant_resources_df['Number of Respondents'].sum()
resources_response_ratio = round(quant_resources_df['Response Ratio'].mean(), 2)
resources_positivity = round(quant_resources_df['Positivity Measure(%)'].mean(), 2)
resources_sd = round(quant_resources_df['Standard Deviation'].mean(), 2)

lss_quant_resources_df = resource_df.loc[less_quantitative]
lss_resources_respondents = lss_quant_resources_df['Number of Respondents'].sum()
lss_resources_response_ratio = round(lss_quant_resources_df['Response Ratio'].mean(), 2)
lss_resources_positivity = round(lss_quant_resources_df['Positivity Measure(%)'].mean(), 2)
lss_resources_sd = round(lss_quant_resources_df['Standard Deviation'].mean(), 2)

resources_comparison = pd.DataFrame({
    'Quantitative': {'Positivity Measure(%)': resources_positivity, 'Standard Deviation': resources_sd,
                     'Response Ratio': resources_response_ratio, 'Number of Respondents': resources_respondents},
    'Less Quantitative': {'Positivity Measure(%)': lss_resources_positivity, 'Standard Deviation': lss_resources_sd,
                          'Response Ratio': lss_resources_response_ratio, 'Number of Respondents': lss_resources_respondents}
})

# For Voice DataFrame
quant_voice_df = voice_df.loc[quantitative]
voice_respondents = quant_voice_df['Number of Respondents'].sum()
voice_response_ratio = round(quant_voice_df['Response Ratio'].mean(), 2)
voice_positivity = round(quant_voice_df['Positivity Measure(%)'].mean(), 2)
voice_sd = round(quant_voice_df['Standard Deviation'].mean(), 2)

lss_quant_voice_df = voice_df.loc[less_quantitative]
lss_voice_respondents = lss_quant_voice_df['Number of Respondents'].sum()
lss_voice_response_ratio = round(lss_quant_voice_df['Response Ratio'].mean(), 2)
lss_voice_positivity = round(lss_quant_voice_df['Positivity Measure(%)'].mean(), 2)
lss_voice_sd = round(lss_quant_voice_df['Standard Deviation'].mean(), 2)

voice_comparison = pd.DataFrame({
    'Quantitative': {'Positivity Measure(%)': voice_positivity, 'Standard Deviation': voice_sd,
                     'Response Ratio': voice_response_ratio, 'Number of Respondents': voice_respondents},
    'Less Quantitative': {'Positivity Measure(%)': lss_voice_positivity, 'Standard Deviation': lss_voice_sd,
                          'Response Ratio': lss_voice_response_ratio, 'Number of Respondents': lss_voice_respondents}
})





In [64]:
teach_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),81.63,90.55
Standard Deviation,2.63,4.55
Response Ratio,0.65,0.74
Number of Respondents,641.00,629.00


In [65]:
learn_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),75.90,84.22
Standard Deviation,2.87,5.51
Response Ratio,0.65,0.74
Number of Respondents,641.00,629.00


In [66]:
assess_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),65.70,78.68
Standard Deviation,3.13,5.94
Response Ratio,0.65,0.74
Number of Respondents,641.00,629.00


In [67]:
support_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),83.80,91.85
Standard Deviation,2.50,4.38
Response Ratio,0.65,0.74
Number of Respondents,641.00,629.00


In [68]:
organisation_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),79.43,82.64
Standard Deviation,2.73,5.91
Response Ratio,0.65,0.74
Number of Respondents,641.00,629.00


In [69]:
resources_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),85.60,88.39
Standard Deviation,2.33,4.82
Response Ratio,0.65,0.74
Number of Respondents,636.00,628.00


In [70]:
voice_comparison

,Quantitative,Less Quantitative
Positivity Measure(%),64.53,72.96
Standard Deviation,3.17,6.45
Response Ratio,0.65,0.74
Number of Respondents,637.00,625.00


As suspected, students studying quantitative degrees were less satisfied in *every* metric tested. That said, the Less Quantitative students had a wider standard deviation for their answers and a students from Quantitative subjects had a lower response ratio (though larger numbers of respondents).

We now turn to answering how LSE student satisfaction has evolved over the years- which requires many pieces of data from the Data folder.The data is arranged in different ways from the 2023 NSS data, which means it requires different cleaning methods.

In [71]:
dat_2015=pd.read_excel('./data/2015_dat.xlsx', sheet_name='NSSFULLTIME1', header=3)
dat_2016=pd.read_excel('./data/2016_dat.xlsx', sheet_name='NSSFULLTIME1', header=3)
dat_2017=pd.read_excel('./data/2017_dat.xlsx', sheet_name='NSSFULLTIME1', header=3)
dat_2018=pd.read_excel('./data/2018_dat.xlsx', sheet_name='NSSFULLTIME1', header=3)
dat_2019=pd.read_excel('./data/2019_dat.xlsx',sheet_name='NSSFULLTIME1', header=3)
dat_2020=pd.read_excel('./data/2020_dat.xlsx',sheet_name='NSSFULLTIME1', header=3)
dat_2021=pd.read_excel('./data/2021_dat.xlsx',sheet_name='NSSFULLTIME1', header=3)
dat_2022=pd.read_excel('./data/2022_dat.xlsx',sheet_name='NSSFULLTIME1', header=3)

In [72]:
dat_2023=overall_df[:1] #the 2023 data is already stored as the overall_df from earlier, when comparing universities
dat_2023=dat_2023.drop('Response Ratio', axis=1) #Response Ratio proved difficult to calculate for other years
dat_2023=dat_2023.transpose()
dat_2023=dat_2023.rename(columns={'LSE': 2023})
dat_2023 #we want to create a dataframe as such for every year which we will then join together

,2023
Teaching,86.5
Learning Opportunities,79.4
Assessment and Feedback,71.1
Academic Support,87.4
Organisation and Management,80.7
Learning Resources,87.1
Student Voice,67.2


The data before 2023 has a different structure. So we explore it a little to decide how to form our dataframes. 

In [73]:
dat_2018.head()

,UKPRN,Provider,Subject Code,Subject,Level,Question Number,Answered 1,Answered 2,Answered 3,Answered 4,Answered 5,N/A,Confidence interval - min,Actual value,Confidence interval - max,Response,Sample Size,Two years aggregate data?
0,10007783,University of Aberdeen,CAH01,Medicine and dentistry,First degree,Q01,0.0000,0.0270,0.0090,0.4955,0.4685,0,0.9030,0.9640,0.9872,111,161,NaN
1,10007783,University of Aberdeen,CAH01,Medicine and dentistry,First degree,Q02,0.0182,0.0273,0.0545,0.3636,0.5364,1,0.8206,0.9000,0.9465,110,161,NaN
2,10007783,University of Aberdeen,CAH01,Medicine and dentistry,First degree,Q03,0.0000,0.0000,0.0450,0.1532,0.8018,0,0.8907,0.9550,0.9822,111,161,NaN
3,10007783,University of Aberdeen,CAH01,Medicine and dentistry,First degree,Q04,0.0000,0.0360,0.0811,0.3063,0.5766,0,0.8006,0.8829,0.9340,111,161,NaN
4,10007783,University of Aberdeen,CAH01,Medicine and dentistry,First degree,Q05,0.0000,0.0270,0.0360,0.3694,0.5676,0,0.8670,0.9369,0.9713,111,161,NaN


We remove unneccessary columns from every dataframe (before 2023, which is already clean).

In [74]:
dataframes=[dat_2015,dat_2016,dat_2017, dat_2018, dat_2019, dat_2020, dat_2021, dat_2022]
columns_to_drop = [
        "UKPRN",
        "Level",
        "Confidence interval - min",
        "Actual value",
        "Confidence interval - max", 
        "N/A",
        "Answered 1",
        "Answered 2",
        "Answered 3"        
    ]

for df in dataframes:
    df.drop(columns_to_drop, axis=1, inplace= True)


dat_2022.head()
    


,Provider,Subject Code,Subject,Question Number,Answered 4,Answered 5,Response,Sample Size
0,University of Aberdeen,CAH01,Medicine and dentistry,Q01,0.5217,0.3913,138,184
1,University of Aberdeen,CAH01,Medicine and dentistry,Q02,0.5435,0.3478,138,184
2,University of Aberdeen,CAH01,Medicine and dentistry,Q03,0.2774,0.6642,137,184
3,University of Aberdeen,CAH01,Medicine and dentistry,Q04,0.3768,0.5000,138,184
4,University of Aberdeen,CAH01,Medicine and dentistry,Scale01,0.4293,0.4746,138,184


It seems data after 2018 includes a Scale01 variable, which calculates the average response for each topic (questions are grouped into topics, e.g. Questions 1 to 4 are to do with teaching). However, 2018 does not include this variable so we will have to calculate the average response for each topic ourselves. 

Additionally, none of the DataFrames have a positivity measure, which is the percentage of students who answered 'very good' or 'good' to a question (or the equivalent). This can easily be calculated by summing the 'Answered 4' and 'Answered 5' categories percentages together.

In [ ]:
years1={2015: dat_2015, 2016: dat_2016, 2017:dat_2017, 2018:dat_2018}
mix={}

for year, dat in years1.items():
    #before 2018 Provider is called Institution
    if year==2018:
        dat=dat[dat['Provider']=='The London School of Economics and Political Science']
    elif year==2014:
        dat=dat[dat['Institution']=='London School of Economics and Political Science']
    else:
        dat=dat[dat['Institution']=='The London School of Economics and Political Science']
    teachdat=dat[(dat["Question Number"]=='Q01')| (dat["Question Number"]=='Q02') |(dat["Question Number"]=='Q03') 
                        | (dat["Question Number"]=='Q04') ]
    learndat=dat[(dat["Question Number"]=='Q05')| (dat["Question Number"]=='Q06') |(dat["Question Number"]=='Q07')]
    assessdat=dat[(dat["Question Number"]=='Q08')| (dat["Question Number"]=='Q09') |(dat["Question Number"]=='Q10') 
                            | (dat["Question Number"]=='Q11')]
    supportdat=dat[(dat["Question Number"]=='Q12')| (dat["Question Number"]=='Q13') |(dat["Question Number"]=='Q14')]
    orgdat=dat[(dat["Question Number"]=='Q15')| (dat["Question Number"]=='Q16') |(dat["Question Number"]=='Q17') ]
    resourcedat=dat[(dat["Question Number"]=='Q18')| (dat["Question Number"]=='Q19') |(dat["Question Number"]=='Q20')]
    voicedat=dat[(dat["Question Number"]=='Q23')| (dat["Question Number"]=='Q24') |(dat["Question Number"]=='Q25') ] 
    #we skipped over Learning community and Student union, as there is no equivalent in 2023
    columns_to_average = ['Answered 4', 'Answered 5']

    # Calculate the mean along the columns axis (axis=1)
    teach=teachdat[columns_to_average].mean().sum()*100
    learn=learndat[columns_to_average].mean().sum()*100
    assess=assessdat[columns_to_average].mean().sum()*100
    support=supportdat[columns_to_average].mean().sum()*100
    org=orgdat[columns_to_average].mean().sum()*100
    resource=resourcedat[columns_to_average].mean().sum()*100
    voice=voicedat[columns_to_average].mean().sum()*100

    mix[year]={'Teaching': teach, 'Learning Opportunities':learn, 'Assessment and Feedback':assess,
                     'Academic Support': support, 'Organisation and Management': org, 'Learning Resources': resource,
                     'Student Voice':voice}

before2019=pd.DataFrame(mix)
before2019=before2019.round(1)
before2019

Now, let's create the other dataframes; 2019-2022.

In [76]:
years={2019:dat_2019, 2020: dat_2020, 2021: dat_2021, 2022: dat_2022}
mixdict={}

for year, dat in years.items():
     dat=dat[dat['Provider']=='The London School of Economics and Political Science']
     teachdat=dat[dat['Question Number']=='Scale01']
     learndat=dat[dat['Question Number']=='Scale02']
     assessdat=dat[dat['Question Number']=='Scale03']
     supportdat=dat[dat['Question Number']=='Scale04']
     orgdat=dat[dat['Question Number']=='Scale05']
     resourcedat=dat[dat['Question Number']=='Scale06']
     voicedat=dat[dat['Question Number']=='Scale08']
    
     columns_to_average = ['Answered 4', 'Answered 5']

     # Calculate the mean along the columns axis (axis=1)
     teach=teachdat[columns_to_average].mean().sum()*100
     learn=learndat[columns_to_average].mean().sum()*100
     assess=assessdat[columns_to_average].mean().sum()*100
     support=supportdat[columns_to_average].mean().sum()*100
     org=orgdat[columns_to_average].mean().sum()*100
     resource=resourcedat[columns_to_average].mean().sum()*100
     voice=voicedat[columns_to_average].mean().sum()*100

     mixdict[year]= {'Teaching': teach, 'Learning Opportunities':learn, 'Assessment and Feedback':assess,
                     'Academic Support': support, 'Organisation and Management': org, 'Learning Resources': resource,
                     'Student Voice':voice}

before2023=pd.DataFrame(mixdict)
before_2023=before2023.round(1)
before_2023

,2019,2020,2021,2022
Teaching,80.1,84.6,82.3,83.5
Learning Opportunities,74.6,78.0,75.8,79.1
Assessment and Feedback,69.3,72.2,68.2,69.7
Academic Support,71.0,76.3,74.1,75.5
Organisation and Management,78.0,82.9,77.1,78.2
Learning Resources,79.4,85.3,79.1,83.2
Student Voice,64.3,69.3,63.7,70.0


We now need to join all the dataframes together.

In [77]:
#As the years 2015-2016 do not have Student Voice, we will remove that
#Learning Opportunities and Learning Resources are also not 
years_df = pd.concat([before2019, before_2023, dat_2023], axis=1)
years_df=years_df.drop('Student Voice')
years_df

,2015,2016,2017,2018,2019,2020,2021,2022,2023
Teaching,82.3,81.5,77.4,74.6,80.1,84.6,82.3,83.5,86.5
Learning Opportunities,70.4,64.3,71.2,68.4,74.6,78.0,75.8,79.1,79.4
Assessment and Feedback,70.0,62.7,61.8,61.6,69.3,72.2,68.2,69.7,71.1
Academic Support,77.4,63.9,65.3,66.5,71.0,76.3,74.1,75.5,87.4
Organisation and Management,86.4,83.5,73.4,73.9,78.0,82.9,77.1,78.2,80.7
Learning Resources,72.8,67.6,80.3,75.3,79.4,85.3,79.1,83.2,87.1


It seems student satisfaction has been on an upward trajectory, though to get a better understanding of the changes we will have to visualise these years. We now move to understanding how LSE is perceived online, through data from Reddit.com.

### Reddit LSE data

A refresh of the format of the data frame used to store this data:

In [78]:
lse_reddit_data_df = pd.read_csv("Data/reddit_data.csv")
lse_reddit_data_df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/reddit_data.csv'

The first thing to check is the number of duplicate or null values. As you can see below, there is neither for both.
- There are no null values since the code used to access the top comment from each post was written to only add the post to the dataframe if it contained a comment. 

- There are no duplicate posts since the chance of two identical strings of words is highly unlikely.

In [ ]:
num_duplicates = lse_reddit_data_df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

In [ ]:
total_nulls = lse_reddit_data_df.isnull().sum().sum()
print(f"Number of null values: {total_nulls}")
print(lse_reddit_data_df.shape)

We only want to consider posts that have a high amount of upvotes, since they are more likely to be helpful and insightful information if other users liked them. We can acheive this by removing posts whose score is below '30', of which there way only one. This number was decided given the maximum score was 327 and the mean score was 97.

It is worth noting that the posts are already sorted in order of descending score, since that is a paramter included in the Reddit API call.

In [ ]:
print(f"Maximum score: {lse_reddit_data_df['Score'].max()}")
print(f"Mean score: {int(lse_reddit_data_df['Score'].mean())}")
print(f"Current number of posts: {lse_reddit_data_df.shape[0]}")
num_low_score_posts = (lse_reddit_data_df['Score']<= 30).sum()
print(f"Number of posts with a score of 5 or below: {num_low_score_posts}")
lse_reddit_data_df = lse_reddit_data_df[lse_reddit_data_df['Score'] > 30]
print(f"New number of posts: {lse_reddit_data_df.shape[0]}")


lse_reddit_data_df.head(5)

### Reddit COVID-19 Data

Similar to the procedure above, the below dataframe about reddit posts that contain the word 'covid' is imported from a csv file and saved as a dataframe.

In [ ]:
covid_reddit_data_df = pd.read_csv("Data/covid_reddit_data.csv")
covid_reddit_data_df.head(5)

To clean the data frame, rows where the value in the 'Top Comment' column was '[deleted]' are removed as this was considered a null value, and where the score is 0.


In [ ]:
#remove null values
covid_reddit_data_df = covid_reddit_data_df[covid_reddit_data_df['Top Comment'] != '[deleted]']

#find summary statistics, remove low  scores
print(f"Maximum score: {covid_reddit_data_df['Score'].max()}")
print(f"Mean score: {int(covid_reddit_data_df['Score'].mean())}")
print(f"Current number of posts: {covid_reddit_data_df.shape[0]}")
num_low_score_posts = (covid_reddit_data_df['Score']== 0).sum()
print(f"Number of posts with a score of 0: {num_low_score_posts}")
covid_reddit_data_df = covid_reddit_data_df[covid_reddit_data_df['Score'] > 0]
print(f"New number of posts: {covid_reddit_data_df.shape[0]}")

#display the full comment in the 'Top Comment' column
pd.set_option('display.max_colwidth', None)

covid_reddit_data_df.head(5)

The next step was to extract any information about how COVID-19 impacted universtity students in London. The results were as follows:

Only 3/200 posts mentioned the words in the 'keywords' list below that were linked to the city of London, or the names of the biggest universities in London. Of the three posts outputted, only one details the impact of COVID-19 on student life by discussing online learning in the King's College London Physics degree. However, they are descirbing the benefits of this course due to it having compulsory in-person labs, whereas most posts in the previous dataframe have a negative view.

The scores of these posts are also all lower than the mean score of 117, signifying there are not as relevant compared to other posts about COVID-19.

The lack of results could be explained by the following reasons:

- Whether a university was located in London had no impact on the student satisfaction during/after COVID-19.
- Students chose not to mentioned the university they attended when posting on the website.
- The information was not accurately gathered as there are too many variations of university names or other
identifying phrases of a student being located in London.
- The sample size was to small.



In [ ]:
keywords = [' lse ', ' ucl ', 'london', ' kcl ', 'kings', ' icl ', 'imperical',
            'imperical', ' ual ', ' uol ', 'university of london ', 'university of west london',
            'university of east london', 'london metropolitan university', ' london met', 'university of greenwhich',
            'university of westminster', 'middlesex university', 'london southbank',
            "regent's university", 'university of roehampton', 'ravensbourne university', 'royal college of art',' rcoa ',
            'birkbeck', ' bpp university', 'guildhall school', 'royal college of music', ' lbs ', 'london business school',
            'rada']

#find posts containing any keywords
london_filtered_df = covid_reddit_data_df[covid_reddit_data_df['Title'].str.contains('|'.join(keywords), case=False, na=False) |
                 covid_reddit_data_df['Top Comment'].str.contains('|'.join(keywords), case=False, na=False)]

london_filtered_df

### Reddit data regarding LSE, UCL, Kings, and Imperial's sentiment analysis

all comments and visualisation are temporary. only the data is polished (ish) stuff. could be optimised further

Data obtained through reddit regarding the online perception of LSE is useful, as it may highlight areas for improvement or key strengths. However, with data regarding LSE's perception, it's hard to gauge how LSE is perceived online as people may act differently online, for example, they may be more negative generally due to the anonymity.

Therefore, it's important to put this data into context, by comparing it with data from other universities. The universities we shall be comparing it to are UCL, Kings, and Imperial. This is because these universities are all in London, a factor which can't be changed about LSE, and they are all considered elite universities, generally referred to as the big four London unversities.

We are limiting the comparisons to these 3 other universities only, for ease of collection and analysis, and they all share similar problems which relate to their location and elite reputation, which can't be easily overcome if at all.

1)Do graph representation and visual statistics showing differences between LSE vs Imperial vs Kings vs UCL sentiment analysis scores online

2)Perform a wordcloud for LSE vs Imperial vs Kings vs UCL, to show the differences in words most commonly associated with those univerisities.

The below code provides the functions which shall be used, to make an API request to openAI to perform sentiment analysis on the reddit titles, and comments, and also to filter out non-informative words from these titles and comments for the word cloud.

In [ ]:
with open('Data/keys.json') as f:
    keys = json.load(f)
API_KEY = keys['openAI']


from openai import OpenAI


client = OpenAI(api_key = API_KEY)

In [ ]:
def analyze_sentiment(comment, university):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a helpful assistant analyzing the positivity or negativity of reddit comments regarding the student satisfaction of {university}, rate each comment between -1 and 1, -1 represents most negative, 0 represents neutral, and 1 represents most positive. Only reply with the number, do not reply with any words under any circumstace"},
            {"role": "user", "content": comment},
        ]
    )
    return response.choices[0].message.content

def filter_text(comment, university):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a helpful assistant assisting in creating a word cloud graph about student satisfaction regarding {university}, from reddit comments. Please remove any words from the comments which don't provide any information regarding student satisfaction about {university}, the comments regarding student satisfaction and experience surrounding {university} can be neutral, negative, positive. Please return the words in a string, with each word separated by a whitespace character."},
            {"role": "user", "content": comment},
        ]
    )
    return response.choices[0].message.content


Below, we will go through the dataframes for LSE, Kings, UCL and Imperial, use the openAI API to generate sentiment analysis scores for each title and comment, and store these back into another dataframe. We must filter the reddit data in the same way we did to the LSE reddit data.

In [ ]:
kings_reddit_data_df = pd.read_csv("Data/kings_reddit_data.csv")
ucl_reddit_data_df = pd.read_csv("Data/ucl_reddit_data.csv")
imperial_reddit_data_df = pd.read_csv("Data/imperial_reddit_data.csv")


other_london_reddit_data_df = [kings_reddit_data_df, ucl_reddit_data_df, imperial_reddit_data_df]


for df in other_london_reddit_data_df:
    df = df[df['Score'] > 30]

In [ ]:
dataframes = [lse_reddit_data_df, kings_reddit_data_df, ucl_reddit_data_df, imperial_reddit_data_df]
sentiment_analysis_dfs = [lse_sentiment_analysis_df, kings_sentiment_analysis_df, ucl_sentiment_analysis_df, imperial_sentiment_analysis_df]
university_names_api = ['the London School of Economics (LSE)', 'Kings College London (Kings)', 'University College London (UCL)', 'Imperial College London (Imperial)']


for dataframe, sentiment_analysis_df, name in zip(dataframes, sentiment_analysis_dfs, university_names_api):
    sentiment_analysis_df = pd.DataFrame(columns=['Title', 'Title_Sentiment_Score', 'Top_Comment', 'Top_Comment_Sentiment_Score'])
    for index, row in dataframe.iterrows():
        title_sentiment = analyze_sentiment(row[0], name)
        comment_sentiment = analyze_sentiment(row[2], name)
        sentiment_analysis_df.loc[index] = [row[0], title_sentiment, row[2], comment_sentiment]

In [ ]:
lse_sentiment_analysis_df

In [ ]:
kings_sentiment_analysis_df

In [ ]:
ucl_sentiment_analysis_df

In [ ]:
imperial_sentiment_analysis_df

In [ ]:
# imperial_sentiment_analysis_df.drop(6, inplace=True)

In [ ]:
imperial_sentiment_analysis_df.reset_index(drop=True, inplace=True)

In [ ]:
imperial_sentiment_analysis_df

In [ ]:
def clean_sentiment_data(input_df):
    cleaned_df = pd.DataFrame(columns=['Title_Sentiment_Score', 'Top_Comment_Sentiment_Score'])
    for index, value in input_df.iterrows():
        cleaned_df.loc[index] = [float(input_df.iloc[index][1]), float(input_df.iloc[index][3])]
    return cleaned_df

In [ ]:
lse_sentiment_cleaned = clean_sentiment_data(lse_sentiment_analysis_df)
kings_sentiment_cleaned = clean_sentiment_data(kings_sentiment_analysis_df)
ucl_sentiment_cleaned = clean_sentiment_data(ucl_sentiment_analysis_df)
imperial_sentiment_cleaned = clean_sentiment_data(imperial_sentiment_analysis_df)


For ease of convenience, and to maximise the available data possible, we shall incorporate both the sentiment score for the title and the top comment.

In [ ]:
lse_sentiment_cleaned_comb = pd.concat([lse_sentiment_cleaned['Title_Sentiment_Score'], lse_sentiment_cleaned['Top_Comment_Sentiment_Score']])
kings_sentiment_cleaned_comb = pd.concat([kings_sentiment_cleaned['Title_Sentiment_Score'], kings_sentiment_cleaned['Top_Comment_Sentiment_Score']])
ucl_sentiment_cleaned_comb = pd.concat([ucl_sentiment_cleaned['Title_Sentiment_Score'], ucl_sentiment_cleaned['Top_Comment_Sentiment_Score']])
imperial_sentiment_cleaned_comb = pd.concat([imperial_sentiment_cleaned['Title_Sentiment_Score'], imperial_sentiment_cleaned['Top_Comment_Sentiment_Score']])

In [ ]:
lse_sentiment_cleaned_comb.head(5)

In [ ]:
kings_sentiment_cleaned_comb.head(5)

In [ ]:
ucl_sentiment_cleaned_comb.head(5)

In [ ]:
imperial_sentiment_cleaned_comb.head(5)

## WARNING: THE FOLLOWING CODE TAKES A LONG TIME TO RUN!

In [ ]:
#Please note that the following code takes about 5-10 minutes to run, as it filters through many 
#reddit comments for 4 universities, it will inform you of its progress as it runs.


print('Please note that the following code takes about 5-10 minutes to run, it will inform you each time one of the four dataframes has been filtered')


lse_filtered_text = set()
kings_filtered_text = set()
ucl_filtered_text = set()
imperial_filtered_text = set()
uni_filtered_texts = [lse_filtered_text, kings_filtered_text,ucl_filtered_text, imperial_filtered_text]
university_names_api = ['the London School of Economics (LSE)', 'Kings College London (Kings)', 'University College London (UCL)', 'Imperical College London (Imperial)']


i = 1
for dataframe, uni_filtered_text, name in zip(dataframes, uni_filtered_texts, university_names_api):
    
    for index, row in dataframe.iterrows():
        title_filtered = filter_text(row[0], name)
        comment_filtered = filter_text(row[2], name)
        uni_filtered_text.update(title_filtered.split())
        uni_filtered_text.update(comment_filtered.split())
        
    print(f'Dataframe {i}/4 complete')
    i += 1


lse_word_cloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(lse_filtered_text))
kings_word_cloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(kings_filtered_text))
ucl_word_cloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(ucl_filtered_text))
imperial_word_cloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(imperial_filtered_text))

# Data analysis and Visualisation

### How does LSE student satisfaction compare to other universities? (Using NSS Data)


Let's first remember the dataframe we created to answer this question.

In [ ]:
overall_df #universities

### How does LSE student satisfaction compare to other universities? (Using Reddit Data)

Firstly, we shall view the summary statistics for the sentiment scores for each of the four universities to be compared.

In [ ]:
kings_sentiment_cleaned_comb.describe()

In [ ]:
ucl_sentiment_cleaned_comb.describe()

In [ ]:
imperial_sentiment_cleaned_comb.describe()

In [ ]:
lse_sentiment_cleaned_comb.describe()

blah blah blah blah. blah blah blah blah. blah blah blah. thats what the summary statistics show : )

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot([lse_sentiment_cleaned_comb, kings_sentiment_cleaned_comb, ucl_sentiment_cleaned_comb, imperial_sentiment_cleaned_comb], labels=['LSE', 'Kings', 'UCL', 'Imperial'])
plt.title('Comparison of Sentiment Scores from London Universities')
plt.xlabel('University')
plt.ylabel('Sentiment score')
plt.grid(True)
plt.show()

BLAH BLAH BLAH, TALK ABOUT THE ABOVE BLAH BLAH BLAH BLAH BLAH BLAH BLAH BLAH BLAH BLABH lbahbds bdkah lbl ablah blahb albh alb halbl blah blahblabh alb al

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].hist(lse_sentiment_cleaned_comb, bins=10, alpha=0.5)
axs[0, 0].set_title('LSE')
axs[0, 0].set_xlabel('Sentiment Score')
axs[0, 0].set_ylabel('Frequency')

axs[0, 1].hist(kings_sentiment_cleaned_comb, bins=10, alpha=0.5)
axs[0, 1].set_title('Kings')
axs[0, 1].set_xlabel('Sentiment Score')
axs[0, 1].set_ylabel('Frequency')


axs[1, 0].hist(ucl_sentiment_cleaned_comb, bins=10, alpha=0.5)
axs[1, 0].set_title('UCL')
axs[1, 0].set_xlabel('Sentiment Score')
axs[1, 0].set_ylabel('Frequency')

axs[1, 1].hist(imperial_sentiment_cleaned_comb, bins=10, alpha=0.5)
axs[1, 1].set_title('Imperial')
axs[1, 1].set_xlabel('Sentiment Score')
axs[1, 1].set_ylabel('Frequency')

plt.tight_layout()

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(lse_word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(kings_word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(imperial_word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(ucl_word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### How does student satisfaction in LSE vary by degree?

Once again, let's remind ourselves of the Dataframe we created to answer this question.

In [ ]:
overall_subjectdf

#Teaching, Learning Opportunities, Academic Support, Organisation and Management
#teach_df, learn_df, support_df, org_df

### How has student satisfaction at the LSE changed over time?

Below, is the years_df we will use to answer this question.

In [ ]:
years_df

In [ ]:
transposed_df = years_df.transpose()

# Plot
plt.figure(figsize=(10, 6))
for column in transposed_df.columns:
    plt.plot(transposed_df.index, transposed_df[column], marker='o', label=column)

plt.title('Academic Metrics Over Years')
plt.xlabel('Year')
plt.ylabel('Positivity Percentage (%)')
plt.grid(True)
plt.tight_layout()

# Display the legend separately
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

Though it seems clear that student satisfaction has been rising over the years in most categories (also remember, that the questions for 2023 had a slightly different format), we  want to get a better idea of the general trend by plotting a line of best fit for each category. Also, since we are dealing with percentages it seems a bit unwise to use this to predict the future years student satisfaction scores. 

In [ ]:
years = years_df.columns.astype(int).values

# Initialize a plot
plt.figure(figsize=(10, 6))

# Loop through each category
for category in years_df.index:
    # Extract the data for the current category
    category_data = years_df.loc[category].values
    
    # Perform linear regression
    slope, intercept, _, _, _ = linregress(years, category_data)
    
    # Predict values for all years using the regression line
    predicted_values = slope * years + intercept
    
    
    # Plot the line of best fit
    plt.plot(years, predicted_values, label=f'{category} Fit')

plt.xlabel('Year')
plt.ylabel('Positivity Percentage (%)')
plt.title('Line of Best Fit for Categories')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.show()

So, it seems as if student satisfaction is rising in most categories, though there are huge fluctuations year to year and so the future is uncertain. The plots also show interesting details, like the fact that through all of the years LSE students are least happy with 'Assessment and Feedback' topic.

# Conclusion

talk about our final results and conclusions from data, and how they answer our questions, or any other additional insight.

Also talk about problems or potential problems with our data collection, cleansing, and results as a result